# <center> Computer Assignment 1 - AI Search </center>
##  In this project, we want to work on some uninformed algorithms like BFS and IDS and also some informed algorithms like $A^*$.<br><br> **Uninformed Search** algorithms are the strategies that use only the information available in the problem definition.<br><br>**Informed Search** algorithms use some hints about the desirability of different states too.



### **Problem Description:** We have a map with n rows and m columns, with one agent at the start point(0, 0) <br> There are some foods and medicines on the map too. Also there are some blocked cells that the agents can't go to. If an agent goes to a cell with a medicine on it, another agent will appear in the cell (n-1, 0). <br>When agents go to a cell, they should eat whatever in that cell is. (either food or medicine)<br>What the problem asks us, is to find an optimal way to eat all the foods and also take all the agents to the endpoint(n-1, m-1)

***
### In this project we are going to:
- Defining the terms of Agent and State in AI search strategies
- Implementing BFS
- Implementing IDS
- Defining huristic function
- Defining an evaluation function
- Implementing $A^*$

***
## Importing the required libraries

In [1]:
import copy
import time
import math
import heapq # min heap used in A-star

***
## Reading the inputs from the file
#### You only need to give the name of your test file that is in the Tests folder.

In [2]:
def read_input(file_name):
    lines = []
    with open(file_name) as test:
        while (line := test.readline().rstrip()):
            lines.append(line)
            
    return lines

In [3]:
filename = input()
path = 'Tests/' + filename
test = read_input(path)

# n: No of rows, m: No of cols
n, m = test[0].split(' ')
n, m = int(n), int(m)
c, k = test[1].split(' ')
c, k = int(c), int(k)
game_map = {}
medicines = []
foods = []

for i in range(n):
    for j in range(m):
        game_map[(i,j)] = 'e'
        
for i in range(c):
    x, y = test[2+i].split(' ')
    x, y = int(x), int(y)
    game_map[(x,y)] = 'food'
    foods.append((x,y))
    
for i in range(k):
    x, y = test[2+c+i].split(' ')
    x, y = int(x), int(y)
    game_map[(x,y)] = 'medicine'
    medicines.append((x, y))
    
# d: No of block nodes
d = int(test[2+c+k])
for i in range(d):
    x, y = test[3+c+k+i].split(' ')
    x, y = int(x), int(y)
    game_map[(x,y)] = 'b'

test2.in


***
#### **Agent** is a class that models our agents with keeping track of their position and id.
- **agent_string()**: used in State class to distinguish the differences between states.

In [4]:
class Agent:
    def __init__(self, x, y, agent_id):
        self.x = x
        self.y = y
        self.agent_id = agent_id
        self.is_finished = False
        
    def agent_string(self):
        return str(self.x) + str(self.y)

***
#### **State** is a class that models our observations about the moves of our agents and eating foods or medicines. To solve the problem, we see different sequences of actions by agents that have some consequences like eating or not eating foods or medicines, we model this sequence of actions with this class.
- **parent:** uses to keep track of the previous states.
- **depth:** shows how far our search goes.
- **check_pos():** checks if the current position that the State saw is medicine or food or none of them. If it was medicine, an agent will be added. That medicine will be deleted too. If it was food, only that food will be deleted.
- **state_string():** as we saw in Agent class, we have a method used to distinguish the differences between states, so we don't see duplicate states in our search algorithms.
- **check_goal():** checks if the current state is our goal state or not.

In [5]:
class State:
    def __init__(self, agents, foods, medicines, parent=None):
        self.agents = agents
        self.parent = parent
        self.medicines = medicines
        self.foods = foods
        if self.parent == None:
            self.depth = 0
        else:
            self.depth = parent.depth + 1
        
    def check_pos(self, x, y, agent_id):
        if game_map[(x, y)] == 'medicine':
            try:
                index = self.medicines.index((x,y))
            except ValueError:
                index = -1
            if index != -1:
                del self.medicines[index]
                new_agent = Agent(n-1, 0, agent_id+1)
                self.agents.append(new_agent)
        
        if game_map[(x, y)] == 'food':
            try:
                index = self.foods.index((x,y))
            except ValueError:
                index = -1
            if index != -1:
                del self.foods[index]
            
    def state_string(self):
        s = ''
        for i in self.agents:
            s += i.agent_string()
        for i in self.foods:
            s += str(i)
        for i in self.medicines:
            s += str(i)
        return s
    
    def check_goal(self):
        if len(self.foods) > 0:
            return False
        for i in self.agents:
            if i.x != n-1 or i.y != m-1:
                return False
        return True

***
## Some Useful stuffs
- **possible_moves:** possible moves for an agent
- **is_valid_pos():** checks if the position is not bigger than the map dimensions and also checks that the position is not blocked.

In [6]:
possible_moves = ['U', 'D', 'R', 'L']

def is_valid_pos(x, y):
    if x >= n or y >= m or x < 0 or y < 0:
        return False
    if game_map[(x, y)] == 'b':
        return False
    return True

***
## Creating initial agent and initial state

In [7]:
agents = []
start_agent = Agent(0, 0, 1)
agents.append(start_agent)
start_state = State(agents, foods, medicines)
start_state.check_pos(0, 0, 1)

***
## BFS
Breadth-first search (BFS) is an algorithm for searching a tree data structure for a node that satisfies a given property. It starts at the tree root and explores all nodes at the present depth prior to moving on to the nodes at the next depth level. Extra memory, usually a queue, is needed to keep track of the child nodes that were encountered but not yet explored.<br>[See the reference](https://en.wikipedia.org/wiki/Breadth-first_search)<br><br>$ \bf BFS \ properties: $
- Complete
- Optimal: because we move in depth
- Time complexity: $\bf O(b^d)$ where **b** is max branching factor of the search tree and **d** is the depth of the optimal solution.
- Space complexity: $ \bf O(b^d) $<br><br>


In [8]:
def BFS(start_state):
    all_states = 0
    fifo = []
    fifo.append(start_state)
    visited = set()
    visited.add(start_state.state_string())

    while(True):
        if len(fifo) <= 0:
            return all_states, state
        state = fifo.pop(0)
        for i in state.agents:
            for move in possible_moves:
                if move == 'U':
                    new_x, new_y = i.x+1, i.y
                if move == 'D':
                    new_x, new_y = i.x-1, i.y
                if move == 'R':
                    new_x, new_y = i.x, i.y+1
                if move == 'L':
                    new_x, new_y = i.x, i.y-1
                if is_valid_pos(new_x, new_y) == True:
                    new_agents = copy.deepcopy(state.agents)
                    new_agents[i.agent_id - 1].x = new_x
                    new_agents[i.agent_id - 1].y = new_y
                    new_foods = copy.deepcopy(state.foods)
                    new_medicines = copy.deepcopy(state.medicines)
                    pre_x, pre_y = state.agents[i.agent_id-1].x, state.agents[i.agent_id-1].y
                    new_state = State(new_agents, new_foods, new_medicines, state)
                    new_state.check_pos(pre_x, pre_y, i.agent_id)
                    all_states += 1
                    if new_state.state_string() not in visited:
                        if new_state.check_goal() == True:
                            return all_states, new_state
                        fifo.append(new_state)
                        visited.add(new_state.state_string())

Here we find all the moves that we can go from a state for all our agents and then check if the new position is valid. If it was a valid position, then we create a new agent and a new state. Finally, we check if we saw the new state before or not.

***
## Execution time, depth, and number of visited states

In [9]:
start = time.time()
all_states, last_state = BFS(start_state)
end = time.time()
print('The execution time is: ', end - start)
print('Goal state is in depth :', last_state.depth)
print('Number of visited states: ', all_states)

The execution time is:  13.241868019104004
Goal state is in depth : 11
Number of visited states:  279371


***
## Printing the path that agents have walked


In [10]:
par = copy.deepcopy(last_state)
moves = []
while(par != None):
    for i in par.agents:
        moves.append((i.agent_id, (i.x, i.y)))
    par = par.parent  
moves = list(dict.fromkeys(moves))

for move in moves[::-1]:
    print('Agent: ', move[0], ' Pos : ', move[1])


Agent:  1  Pos :  (0, 0)
Agent:  1  Pos :  (0, 1)
Agent:  1  Pos :  (0, 2)
Agent:  1  Pos :  (1, 2)
Agent:  1  Pos :  (2, 2)
Agent:  1  Pos :  (3, 2)
Agent:  2  Pos :  (3, 0)
Agent:  2  Pos :  (2, 0)
Agent:  2  Pos :  (2, 1)
Agent:  2  Pos :  (2, 2)
Agent:  2  Pos :  (3, 2)
Agent:  2  Pos :  (3, 3)
Agent:  1  Pos :  (3, 3)


***
## BFS Results


| Test | goal depth | visited states |  time(sec)
|------|------------|----------------|------------
|  1   |     11     |      4304      |    0.135
|  2   |     11     |     279371     |    13.144
|  3   |     19     |     504847     |    21.686


***
## IDS
Iterative deepening search or more specifically iterative deepening depth-first search (IDS or IDDFS) is a state space/graph search strategy in which a depth-limited version of depth-first search is run repeatedly with increasing depth limits until the goal is found. IDDFS is optimal like breadth-first search, but uses much less memory; at each iteration, it visits the nodes in the search tree in the same order as depth-first search, but the cumulative order in which nodes are first visited is effectively breadth-first.<br>[See the reference](https://en.wikipedia.org/wiki/Iterative_deepening_depth-first_search)
<br><br>$ \bf IDS \ properties: $
- Complete
- Optimal (if step cost = 1): because we limit depth of DFS.
- Time complexity: $\bf O(b^d)$ where **b** is max branching factor of the search tree and **d** is the depth of the optimal solution.
- Space complexity: $ \bf O(bd) $
- We can see that IDS uses much less space compared to BFS.

In [11]:
def IDS(state, depth):
    global all_states
    visited.add((state.state_string(), state.depth))
    if state.check_goal() == True:
        print('goal')
        return state
    if depth <= 0:
        return None
    for i in state.agents:
        for move in possible_moves:
            if move == 'U':
                new_x, new_y = i.x+1, i.y
            if move == 'D':
                new_x, new_y = i.x-1, i.y
            if move == 'R':
                new_x, new_y = i.x, i.y+1
            if move == 'L':
                new_x, new_y = i.x, i.y-1
            if is_valid_pos(new_x, new_y) == True:
                new_agents = copy.deepcopy(state.agents)
                new_agents[i.agent_id - 1].x = new_x
                new_agents[i.agent_id - 1].y = new_y
                new_foods = copy.deepcopy(state.foods)
                new_medicines = copy.deepcopy(state.medicines)
                pre_x, pre_y = state.agents[i.agent_id-1].x, state.agents[i.agent_id-1].y
                new_state = State(new_agents, new_foods, new_medicines, state)
                new_state.check_pos(pre_x, pre_y, i.agent_id)
                all_states += 1
                
                if (new_state.state_string(), new_state.depth) not in visited:
                    res = IDS(new_state, depth-1)
                    if res != None:
                        return res
            

Here we recursively go through each depth of our tree and search for the goal state. The other things are similar to the BFS.

***
## Execution time, depth, and number of visited states

In [12]:
depth = 0
start = time.time()
all_states = 0

while True:
    visited = set()
    last_state = IDS(start_state, depth)
    if last_state != None:
        break
    print(depth)
    depth += 1
end = time.time()
print('The execution time is: ', end - start)
print('Goal state is in depth :', last_state.depth)
print('Number of visited states: ', all_states)

0
1
2
3
4
5
6
7
8
9
10
goal
The execution time is:  39.03666925430298
Goal state is in depth : 11
Number of visited states:  832722


***
## Printing the path that agents have walked


In [13]:
par = copy.deepcopy(last_state)
moves = []
while(par != None):
    for i in par.agents:
        moves.append((i.agent_id, (i.x, i.y)))
    par = par.parent  
moves = list(dict.fromkeys(moves))

for move in moves[::-1]:
    print('Agent: ', move[0], ' Pos : ', move[1])


Agent:  1  Pos :  (0, 0)
Agent:  1  Pos :  (0, 1)
Agent:  1  Pos :  (0, 2)
Agent:  1  Pos :  (1, 2)
Agent:  1  Pos :  (2, 2)
Agent:  1  Pos :  (3, 2)
Agent:  2  Pos :  (3, 0)
Agent:  2  Pos :  (2, 0)
Agent:  2  Pos :  (2, 1)
Agent:  2  Pos :  (2, 2)
Agent:  2  Pos :  (3, 2)
Agent:  2  Pos :  (3, 3)
Agent:  1  Pos :  (3, 3)


***
## IDS Results

| Test | goal depth | visited states |  time(sec)
|------|------------|----------------|------------
|  1   |     11     |     24731      |    0.7487
|  2   |     11     |     832722     |    37.180
|  3   |     19     |     2222311    |    91.587

***
### What is Heuristic function?
The **heuristic function** is a way to inform the search about the direction to a goal. It uses an evaluation function to rank states and select the most promising one for expansion. 


### The heuristic function I used
I calculated the distance of all agents to the endpoint(n-1, m-1), then I picked the largest one as my **estimated cost from a state to the goal state** or **heuristic function** $\bf h(n)$.<br>
This heuristic is similar to the **'Manhattan distance'**. (Manhattan distance is calculated as the sum of the absolute differences between the two vectors.)<br>Then I added the depth of the current state as my **cost to reach the current state** or $\bf g(n)$.<br>
So the evaluation function I used is: $ \bf f(n) = g(n) + h(n) $

### Is this heuristic function consistent?
A heuristic $\bf h$ is **consistent** (or monotone) if
- for each node N and each child N’ of N: $h(N) ≤ c(N,N’) + h(N’)$ <br>$c(N,N’)$ is the true cost to go from N to N'.
- for each goal node G: $h(G) = 0$<br>

If these conditions are confirmed, we can say that the $\bf f$ value along a path never decreases and the A-star search is optimal.<br><br>
#### prove:
Because if our agents move(wherever it goes), then $\bf h(current) - \bf h(goal) <= 1$, so this heuristic is consistent.


In [ ]:
def evaluation_function(state, weight=1):
    min_dist = -1
    
    for agent in state.agents:
        if min_dist < abs(agent.x - n) + abs(agent.y - m):
            min_dist = abs(agent.x - n) + abs(agent.y - m)
        
    return min_dist * weight + state.depth

***
### Priority queue
We used priority queue in A-star algorithm, because we need to pop the state that has minimum evaluation function every time. So with this priority queue, we can pop states in O(1). <br>This Node class, helps us to have a priority queue of list of tuples.<br>[See the reference code](https://stackoverflow.com/a/59956131/12576240)


In [ ]:
class Node(object):
    def __init__(self, state, val):
        self.state = state
        self.val = val

    def __lt__(self, other):
        return self.val <= other.val

***
## $\bf A^*$
A* is a graph traversal and path search algorithm, which is often used in many fields of computer science due to its completeness, optimality, and optimal efficiency.<br><br>$ \bf A^* \ properties: $
- Complete
- Optimal: if we have a consistent heuristic.
- Time complexity: number of nodes for which $\bf f(n) <= C^*$ (exponential)
- Space complexity: exponential

<br>In $A^*$, we use **evaluation function** to have some information about where our agents should go. At each iteration of its main loop, $A^*$ needs to determine which of its paths to extend. It does so based on the cost of the path and an estimate of the cost required to extend the path all the way to the goal.<br>
In **weighted** $\bf A^*$ **search**, we take an admissible heuristic, multiply it by an α > 1, and then we perform $ A^*$ search as usual. Fewer nodes tend to get expanded, but the resulting solution may be suboptimal (its cost will be at most α times the cost of the optimal solution)



In [ ]:
def Astar(start_state, weight=1):
    all_states = 0
    min_heap = []
    heapq.heappush(min_heap, Node(start_state, evaluation_function(start_state, weight)))
    
    visited = set()
    visited.add(start_state.state_string())
    
    while(True):
        if len(min_heap) <= 0:
            return state
        state = heapq.heappop(min_heap).state

        for i in state.agents:
            for move in possible_moves:
                if move == 'U':
                    new_x, new_y = i.x+1, i.y
                if move == 'D':
                    new_x, new_y = i.x-1, i.y
                if move == 'R':
                    new_x, new_y = i.x, i.y+1
                if move == 'L':
                    new_x, new_y = i.x, i.y-1

                if is_valid_pos(new_x, new_y) == True:
                    new_agents = copy.deepcopy(state.agents)
                    new_agents[i.agent_id - 1].x = new_x
                    new_agents[i.agent_id - 1].y = new_y
                    new_foods = copy.deepcopy(state.foods)
                    new_medicines = copy.deepcopy(state.medicines)
                    pre_x, pre_y = state.agents[i.agent_id-1].x, state.agents[i.agent_id-1].y
                    new_state = State(new_agents, new_foods, new_medicines, state)
                    new_state.check_pos(pre_x, pre_y, i.agent_id)
                    all_states += 1
                    if new_state.state_string() not in visited:
                        if new_state.check_goal() == True:
                            return all_states, new_state
                        heapq.heappush(min_heap, Node(new_state, evaluation_function(new_state, weight)))
                        visited.add(new_state.state_string())

***
## Execution time, depth, and number of visited states

In [ ]:
start = time.time()
all_states, last_state = Astar(start_state, weight=1)
end = time.time()
print('The execution time is: ', end - start)
print('Goal state is in depth :', last_state.depth)
print('Number of visited states: ', all_states)

***
## Printing the path that agents have walked


In [ ]:
par = copy.deepcopy(last_state)
moves = []
while(par != None):
    for i in par.agents:
        moves.append((i.agent_id, (i.x, i.y)))
    par = par.parent  
moves = list(dict.fromkeys(moves))

for move in moves[::-1]:
    print('Agent: ', move[0], ' Pos : ', move[1])

***
## A* and weighted A* results

| Test |   weight  | goal depth | visited states |  time(sec)
|------|-----------|------------|----------------|------------
|  1   |     1     |     11     |      2201      |    0.08025
|  1   |     1.5   |     11     |      1093      |    0.04404
|  1   |     2     |     11     |      403       |    0.02029
|  2   |     1     |     11     |      15507     |    0.80652
|  2   |     1.5   |     11     |      3374      |    0.17202
|  2   |     2     |     11     |      1976      |    0.10705
|  3   |     1     |     19     |      54540     |    2.58487
|  3   |     1.5   |     19     |      19966     |    0.97051
|  3   |     2     |     19     |      4804      |    0.22370


***
## Final results


#### Test 1:
|                                      | goal depth | visited states  |  mean time(sec)
|--------------------------------------|------------|-----------------|--------------------
|  **BFS**                             |     11     |      4304       |    0.1312236785888
|  **IDS**                             |     11     |      24731      |    0.7816061178843
|$\bf A^*$                             |     11     |      2201       |    0.0751590003967
|**weighted**$$\bf A^* (\alpha=1.5) $$ |     11     |      1093       |    0.0366712280273
|**weighted**$$\bf A^* (\alpha=2  ) $$ |     11     |      403        |    0.0140038562774


#### Test 2:
|                                      | goal depth | visited states |  mean time(sec)
|--------------------------------------|------------|----------------|------------
|  **BFS**                             |     11     |      279371    |    13.074145078659
|  **IDS**                             |     11     |      832722    |    38.386329889297
|$\bf A^*$                             |     11     |      15507     |    0.8199410343170
|**weighted**$$\bf A^* (\alpha=1.5) $$ |     11     |      3374      |    0.1913731098175
|**weighted**$$\bf A^* (\alpha=2  ) $$ |     11     |      1976      |    0.1048440933224



#### Test 3:
|                                      | goal depth | visited states |  mean time(sec)
|--------------------------------------|------------|----------------|------------
|  **BFS**                             |     19     |      50484     |    22.16307020187378
|  **IDS**                             |     19     |      2222311   |    93.60527706146243
|$\bf A^*$                             |     19     |      54540     |    2.51611685752868
|**weighted**$$\bf A^* (\alpha=1.5) $$ |     19     |      19966     |    0.90068387985229
|**weighted**$$\bf A^* (\alpha=2  ) $$ |     19     |      4804      |    0.21028232574462



***
## Conclusion
#### Alogorithms Properties:
If we compare these 3 algorithms based on their properties, we can say all of these algorithms are **optimal** and **complete**.<br>
BFS and IDS have the same **time complexity** but IDS uses much less space compared to BFS.<br>And the time complexity in $A^*$ is the number of nodes which heuristic expands.
#### Time Results:
Using **informed search** strategies can help us to reach the goal state in efficient time compared to **Uninformed search** strategies, because with using informed search, we give our agents some other informations about the world that can help them to make better decisions.
